<a href="https://colab.research.google.com/github/AlexKressner/Vorlesung-Industrielles-Management/blob/main/Produktionsplanung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Produktionsplanung
## Capacitated Lot Sizing Problem (CLSP)

## Modell

### Indexmengen
$p \in P$ : Menge der Produkte

$t \in T$ : Menge der Planungsperioden (Wochen)



### Parameter
$cap_t$ : Kapazität der Abfüllanlage in Woche $t$

$lc_{p}$ : Lagerkosten für Produkt $p$

$rc_{p}$ : Rüstkosten für Produkt $p$

$bz_p$ : Bearbeitungszeit für Produkt $p$

$rz_p$ : Rüstzeit für Produkt $p$

$d_{pt}$ : Nachfrage nach Produkt $p$ in Woche $t$

### Variablen
$Q_{pt} \ge 0$ : Produktionsmenge von Produkt $p$ in Woche $t$

$B_{pt} \ge 0$ : Lagerbestand von Produkt $p$ in Woche $t$

$R_{pt} \in \{0,1\}$ : Binäre Rüstvariable

### Zielfunktion
Min $K = \sum_{p,t} (rc_p * R_{p,t} + lc_p * B_{pt})$

### Nebenbedingungen

**(1) Lagerbilanzgleichung**

$B_{pt} = B_{p,t-1} + Q_{pt} - d_{pt}$

$∀ p, t$

**(2) Produktionskapazitäten**

$\sum_{p} (bz_p * Q_{pt} + rz_p * R_{pt}) \le pcap_{t}$

$∀ t$

**(3) Koppelung Produktions- mit Rüstentscheidung**

$Q_{pt} \le \frac{cap_t}{bz_p} * R_{pt} $

$∀ p, t$

## Implementierung

In [ ]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [ ]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Datenaufbereitung


1.   Problemrelevante Daten in Google-Drive laden
2.   Google-Drive mit Colab-Notebook verbinden
3.   Daten mit `pandas` laden



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Ordner finden
! ls drive/MyDrive/Industrielles_Management/Daten/Produktionsplanung

Erweiterung_Nachfrage.csv   Kosten_Zeiten.csv
Erweiterung_Rüstzeiten.csv  Nachfrage.csv


In [ ]:
# Pfad zurückgeben
! cd drive/MyDrive/Industrielles_Management/Daten/Produktionsplanung && pwd

/content/drive/MyDrive/Industrielles_Management/Daten/Produktionsplanung


In [ ]:
# Daten laden
import pandas as pd

In [ ]:
path = "/content/drive/MyDrive/Industrielles_Management/Daten/Produktionsplanung"

In [ ]:
# Nachfrage
nachfrage = pd.read_csv(f"{path}/Nachfrage.csv", sep=";")

In [ ]:
nachfrage

,Produkt,Woche,Nachfrage
0,Well,1,140
1,Well,2,350
2,Well,3,420
3,Well,4,1400
4,Well,5,0
5,BraTee,1,0
6,BraTee,2,70
7,BraTee,3,560
8,BraTee,4,0
9,BraTee,5,140


In [ ]:
# Kosten & Zeiten
kosten_zeiten = pd.read_csv(f"{path}/Kosten_Zeiten.csv", sep=";", decimal=",")

In [ ]:
kosten_zeiten

,Produkt,Rüstkosten,Lagerkosten,Rüstzeit,Bearbeitungszeit
0,Well,100,3,420,1
1,BraTee,250,5,700,2
2,HafTee,200,6,560,1
3,DirTee,150,4,560,3
4,StraB,250,3,1400,2


## Indexmengen

In [ ]:
P = nachfrage["Produkt"].unique().tolist() # Menge der Produkte

In [ ]:
T = nachfrage["Woche"].unique().tolist() # Menge der Planungsperioden (Wochen)

## Entscheidungsvariablen

In [ ]:
infinity = solver.infinity()

In [ ]:
# Produktionsmengen
Q={}
for p in P:
  for t in T: 
    Q[p,t] = solver.NumVar(0.0, infinity, f"{p},{t}")

In [ ]:
# Lagerbestand
B={}
for p in P:
  for t in T:
    B[p,t] = solver.NumVar(0.0, infinity, f"{p},{t}")

In [ ]:
# Rüstvariablen
R={}
for p in P: 
  for t in T:
        R[p,t] = solver.BoolVar(f"{p},{t}")

In [ ]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 75


## Parameter

In [ ]:
# Kosten
lc = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Lagerkosten"]
rc = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Rüstkosten"]

In [ ]:
# Zeiten
bz = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Bearbeitungszeit"]
rz = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Rüstzeit"]

In [ ]:
# Nachfrage
d = nachfrage.set_index(["Produkt","Woche"]).to_dict("dict")["Nachfrage"]

In [ ]:
# Kapazitäten 
cap = {}
for t in T:
  cap[t] = 5000

## Zielfunktion

### Zielfunktion
Min $K = \sum_{p,t} (rc_p * R_{p,t} + lc_p * B_{pt})$

In [ ]:
# Minimierung der gesamten Kosten
solver.Minimize(sum(rc[p] * R[p,t] + lc[p] * B[p,t] for p in P for t in T))

## Nebenbedingungen

**(1) Lagerbilanzgleichung**

$B_{pt} = B_{p,t-1} + Q_{pt} - d_{pt}$

$∀ p, t$

In [ ]:
for p in P:
  for t in T:
    solver.Add(B[p,t] == B.get((p,t-1),0) + Q[p,t] - d[p,t])

**(2) Produktionskapazitäten**

$\sum_{p} (bz_p * Q_{pt} + rz_p * R_{pt}) \le cap_{t}$

$∀ t$

In [ ]:
for t in T:
  solver.Add(sum(bz[p] * Q[p,t] + rz[p] * R[p,t] for p in P) <= cap[t])

**(3) Koppelung Produktions- mit Rüstentscheidung**

$Q_{pt} \le \frac{cap_t}{bz_p} * R_{pt} $

$∀ p, t$

In [ ]:
for p in P:
  for t in T:
    solver.Add(Q[p,t] <= (cap[t]/bz[p]) * R[p,t])

## Berechnung Lösung

In [ ]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Kosten) =', solver.Objective().Value())
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Kosten) = 12766.666666666664


In [ ]:
for t in T:
  print(f"Woche: {t}")
  produzieren = sum(bz[p] * Q[p,t].solution_value() for p in P)
  rüsten = sum(rz[p] * R[p,t].solution_value() for p in P)
  print(f"Produktionszeit: {round(produzieren)}")
  print(f"Rüstzeit: {round(rüsten)}")
  print(f"Summe: {round(rüsten+produzieren)}")
  
  print("\n")


Woche: 1
Produktionszeit: 1650
Rüstzeit: 3080
Summe: 4730


Woche: 2
Produktionszeit: 3230
Rüstzeit: 1540
Summe: 4770


Woche: 3
Produktionszeit: 3180
Rüstzeit: 1820
Summe: 5000


Woche: 4
Produktionszeit: 3180
Rüstzeit: 1820
Summe: 5000


Woche: 5
Produktionszeit: 3880
Rüstzeit: 1120
Summe: 5000


